In [2]:
from datasets import load_dataset
import ollama



In [3]:
# Load the dataset
dataset = load_dataset("maribr/publication_dates_fr")
train_ds = dataset['train']


Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
# Convert the train dataset to a DataFrame
df = train_ds.to_pandas()
print(df.head())

                                                Text Gold published date  \
0  PROCES-VERBAL DE LA REUNION PUBLIQUE\nDU CONSE...          16/01/2023   
1  CONSEIL COMMUNAUTAIRE DU\n25 JANVIER 2023\nPRO...          25/01/2023   
2  Date de mise en ligne de\nl’acte : 02/ 02/2023...          02/02/2023   
3  Envoyé en préfecture le 26/01/2023\nReçu en pr...          26/01/2023   
4       \nFait à Bourg-en-Bresse, le 23 janvier 2023          16/01/2023   

                                                 url  
0  http://www.ville-saint-ay.fr/userfile/fichier-...  
1  https://www.gatine-racan.fr/wp-content/uploads...  
2  https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf  
3  https://www.fier-et-usses.com/cms_viewFile.php...  
4  https://www.grandbourg.fr/cms_viewFile.php?idt...  


In [13]:
def preprocess_french_text(text):
    """
    Extracts the 5 words before each French-style date in the text and returns the cleaned text.
    Skips processing if the input is None or empty.
    """
    if not text:  # Skip None or empty strings
        return None

    # French month names
    months = r"(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)"

    # Regex pattern for French dates
    date_pattern = fr"(\b\d{{1,2}}(?:er)?\s{months}\s\d{{4}})|(\b\d{{4}}-\d{{2}}-\d{{2}}\b)|(\b\d{{1,2}}/\d{{1,2}}/\d{{4}}\b)"

    # Find all matches
    matches = re.finditer(date_pattern, text, re.IGNORECASE)

    extracted_segments = []

    for match in matches:
        date = match.group(0)  # The date string
        start_idx = match.start()  # Start index of the date in the text

        # Extract 5 words before the date
        before = text[:start_idx].split()[-5:]  # Get the last 5 words before the date
        context = " ".join(before) + " " + date

        extracted_segments.append(context)

    # Join extracted contexts with newline or return None if no matches
    return "\n".join(extracted_segments) if extracted_segments else None

# Process the DataFrame
df['preprocessed_text'] = df['Text'].apply(preprocess_french_text)

# Display the updated DataFrame
print(df['preprocessed_text'].head())

0    LA COMMUNE DE SAINT-AY LUNDI 16 JANVIER 2023\n...
1             CONSEIL COMMUNAUTAIRE DU 25 JANVIER 2023
2    convocation du Conseil Municipal : 13 janvier ...
3    Envoyé en préfecture le 26/01/2023\n26/01/2023...
4           Fait à Bourg-en-Bresse, le 23 janvier 2023
Name: preprocessed_text, dtype: object


In [ ]:

for text in df['Text']:
    messages = [
        {
            'role': 'system',
            'content': (
                "You are an assistant that identifies the most probable publication date of online articles. "
                "Your task is to extract and return the publication date in DD/MM/YYYY format. "
                "If multiple dates are mentioned, prioritize the earliest one that looks like a publication date. "
                "If no date is found, return 'None' with no explanation."
                "you should output only one date"
            )
        },
        {
            'role': 'user',
            'content': text
        }
    ]
    response = chat(model='llama3.2', messages=messages)
    # put the extracted date in a new column
    df['extracted_date'] = response['message']['content']
    print(response['message']['content'])

16/01/2023
25/01/2023
02/02/2023
26/01/2023
23/01/2023
26/02/2023
13/02/2023
07/02/2023
16/02/2023
23/02/2023
06/03/2023
02/02/2023
09/02/2023
15/02/2023
15/02/2023
17/02/2023
16/02/2023
20/02/2023
08/02/2023
Ce qui est à prendre en compte ici, c'est la situation de M. Cordin, le locataire, quant au fait que son bailleur a été condamné par la justice à réparer les préjudices causés par son manque d'entretien du patrimoine immobilier. Le bailleur, qui n'a pas su ou voulu prendre soin de son propriété, a donc créé des problèmes pour M. Cordin et pour l'ensemble des locataires. La décision proposée par M. Jous consiste à faire un moratoire sur les garanties d'emprunt accordées aux bailleurs jusqu'à ce qu'ils signent la charte, qui devrait s'agencer pour réglementer leurs responsabilités envers leurs locataires.
Voici une analyse de l'affaire :

**Contexte**

La commune d'Ermont souhaite ceder un immeuble situé 46, Rue du Général Decaen à Ermont, en zone U23 (habitat collectif) au PLU en v